In [119]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import pandas as pd
import re
import requests
import seaborn as sns
import time
import descartes
import geopandas as gpd

from arcgis.geocoding import geocode
from arcgis.gis import GIS
from dotenv import load_dotenv
from functools import reduce
from shapely.geometry import Point, Polygon
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.svm import SVC, SVR
from textwrap import wrap
from xgboost import XGBClassifier, XGBRegressor, plot_importance
from yellowbrick.classifier import ROCAUC, ConfusionMatrix

plt.style.use("ggplot")
sns.set_context("talk")

In [2]:
load_dotenv("")
censuskey = os.getenv("censuskey")

In [185]:
var_description = pd.DataFrame(pd.DataFrame(
    requests.get("https://api.census.gov/data/2016/acs/acs5/profile/variables.json")
                                            .json()["variables"])
                               .T.iloc[:, 3]
                               .str.title()
                               .str.split("!!", expand=True))

In [189]:
var_description.loc[var_description.index.str.contains(r"PE")].sort_index()

,0,1,2,3,4,5,6
DP02PR_0001PE,Percent,Households By Type,Total Households,None,None,None,None
DP02PR_0002PE,Percent,Households By Type,Total Households,Family Households (Families),None,None,None
DP02PR_0003PE,Percent,Households By Type,Total Households,Family Households (Families),With Own Children Of The Householder Under 18 ...,None,None
DP02PR_0004PE,Percent,Households By Type,Total Households,Family Households (Families),Married-Couple Family,None,None
DP02PR_0005PE,Percent,Households By Type,Total Households,Family Households (Families),Married-Couple Family,With Own Children Of The Householder Under 18 ...,None
DP02PR_0006PE,Percent,Households By Type,Total Households,Family Households (Families),"Male Householder, No Wife Present, Family",None,None
DP02PR_0007PE,Percent,Households By Type,Total Households,Family Households (Families),"Male Householder, No Wife Present, Family",With Own Children Of The Householder Under 18 ...,None
DP02PR_0008PE,Percent,Households By Type,Total Households,Family Households (Families),"Female Householder, No Husband Present, Family",None,None
DP02PR_0009PE,Percent,Households By Type,Total Households,Family Households (Families),"Female Householder, No Husband Present, Family",With Own Children Of The Householder Under 18 ...,None
DP02PR_0010PE,Percent,Households By Type,Total Households,Nonfamily Households,None,None,None


In [14]:
var_description.to_csv("../data/var_description.csv")

In [19]:
variables_selected = var_description.loc[((~var_description.index.str.contains(r'DP')|
                                           (var_description.index.str.contains(r'DP')&
                                            var_description.index.str.contains(r'[0-9]PE')))&
                                          ~var_description.index.str.contains(r'PR_'))].sort_index()

In [109]:
len(variables_selected)

554

In [110]:
resp = requests.get(
        "https://api.census.gov/data/2016/acs/acs5/profile?get={}&for=county:*&in=state:*&key={}"
        .format(",".join(variables_selected.iloc[0:10, :].index), censuskey))

In [111]:
resp.status_code

200

In [96]:
",".join(variables_selected.iloc[10:15, :].index)

'CSA,DIVISION,DP02_0001PE,DP02_0002PE,DP02_0003PE'

In [115]:
resp.status_code

400

In [114]:
reduce(lambda left, right: pd.merge(left, right, how='outer'), varlist_2016_test).shape

(3220, 552)

In [194]:
varlist_2014 = []
k = 0
for var in variables_selected.index:
    resp = requests.get(
        "https://api.census.gov/data/2014/acs/acs5/profile?get={}&for=county:*&in=state:*&key={}"
        .format(var, censuskey))
    if resp.status_code ==200:
        varlist_2014.append(pd.DataFrame(resp.json()[1:], columns=resp.json()[0]))
    k += 1
    if k%int(len(variables_selected)/10)==0:
        print("---progress: {}% completed---".format(k*10/int(len(variables_selected)/10)))

---progress: 10.0% completed---
---progress: 20.0% completed---
---progress: 30.0% completed---
---progress: 40.0% completed---
---progress: 50.0% completed---
---progress: 60.0% completed---
---progress: 70.0% completed---
---progress: 80.0% completed---
---progress: 90.0% completed---
---progress: 100.0% completed---


In [195]:
df_2014 = reduce(lambda left, right: pd.merge(left, right, how='outer'), varlist_2014)
df_2014.shape

(3220, 548)

In [172]:
pd.set_option('display.max_columns', None)
df_2017.loc[df_2017.state!="72"].apply(pd.to_numeric, errors="ignore").describe()

,AIANHH,state,county,ANRC,CBSA,CNECTA,COUNTY,COUSUB,CSA,DIVISION,DP02_0001PE,DP02_0002PE,DP02_0003PE,DP02_0004PE,DP02_0005PE,DP02_0006PE,DP02_0007PE,DP02_0008PE,DP02_0009PE,DP02_0010PE,DP02_0011PE,DP02_0012PE,DP02_0013PE,DP02_0014PE,DP02_0015PE,DP02_0016PE,DP02_0017PE,DP02_0018PE,DP02_0019PE,DP02_0020PE,DP02_0021PE,DP02_0022PE,DP02_0023PE,DP02_0024PE,DP02_0025PE,DP02_0026PE,DP02_0027PE,DP02_0028PE,DP02_0029PE,DP02_0030PE,DP02_0031PE,DP02_0032PE,DP02_0033PE,DP02_0034PE,DP02_0035PE,DP02_0036PE,DP02_0037PE,DP02_0038PE,DP02_0039PE,DP02_0040PE,DP02_0041PE,DP02_0042PE,DP02_0043PE,DP02_0044PE,DP02_0045PE,DP02_0046PE,DP02_0047PE,DP02_0048PE,DP02_0049PE,DP02_0050PE,DP02_0051PE,DP02_0052PE,DP02_0053PE,DP02_0054PE,DP02_0055PE,DP02_0056PE,DP02_0057PE,DP02_0058PE,DP02_0059PE,DP02_0060PE,DP02_0061PE,DP02_0062PE,DP02_0063PE,DP02_0064PE,DP02_0065PE,DP02_0066PE,DP02_0067PE,DP02_0068PE,DP02_0069PE,DP02_0070PE,DP02_0071PE,DP02_0072PE,DP02_0073PE,DP02_0074PE,DP02_0075PE,DP02_0076PE,DP02_0077PE,DP02_0078PE,DP02_0079PE,DP02_0080PE,DP02_0081PE,DP02_0082PE,DP02_0083PE,DP02_0084PE,DP02_0085PE,DP02_0086PE,DP02_0087PE,DP02_0088PE,DP02_0089PE,DP02_0090PE,DP02_0091PE,DP02_0092PE,DP02_0093PE,DP02_0094PE,DP02_0095PE,DP02_0096PE,DP02_0097PE,DP02_0098PE,DP02_0099PE,DP02_0100PE,DP02_0101PE,DP02_0102PE,DP02_0103PE,DP02_0104PE,DP02_0105PE,DP02_0106PE,DP02_0107PE,DP02_0108PE,DP02_0109PE,DP02_0110PE,DP02_0111PE,DP02_0112PE,DP02_0113PE,DP02_0114PE,DP02_0115PE,DP02_0116PE,DP02_0117PE,DP02_0118PE,DP02_0119PE,DP02_0120PE,DP02_0121PE,DP02_0122PE,DP02_0123PE,DP02_0124PE,DP02_0125PE,DP02_0126PE,DP02_0127PE,DP02_0128PE,DP02_0129PE,DP02_0130PE,DP02_0131PE,DP02_0132PE,DP02_0133PE,DP02_0134PE,DP02_0135PE,DP02_0136PE,DP02_0137PE,DP02_0138PE,DP02_0139PE,DP02_0140PE,DP02_0141PE,DP02_0142PE,DP02_0143PE,DP02_0144PE,DP02_0145PE,DP02_0146PE,DP02_0147PE,DP02_0148PE,DP02_0149PE,DP02_0150PE,DP02_0151PE,DP02_0152PE,DP03_0001PE,DP03_0002PE,DP03_0003PE,DP03_0004PE,DP03_0005PE,DP03_0006PE,DP03_0007PE,DP03_0008PE,DP03_0009PE,DP03_0010PE,DP03_0011PE,DP03_0012PE,DP03_0013PE,DP03_0014PE,DP03_0015PE,DP03_0016PE,DP03_0017PE,DP03_0018PE,DP03_0019PE,DP03_0020PE,DP03_0021PE,DP03_0022PE,DP03_0023PE,DP03_0024PE,DP03_0025PE,DP03_0026PE,DP03_0027PE,DP03_0028PE,DP03_0029PE,DP03_0030PE,DP03_0031PE,DP03_0032PE,DP03_0033PE,DP03_0034PE,DP03_0035PE,DP03_0036PE,DP03_0037PE,DP03_0038PE,DP03_0039PE,DP03_0040PE,DP03_0041PE,DP03_0042PE,DP03_0043PE,DP03_0044PE,DP03_0045PE,DP03_0046PE,DP03_0047PE,DP03_0048PE,DP03_0049PE,DP03_0050PE,DP03_0051PE,DP03_0052PE,DP03_0053PE,DP03_0054PE,DP03_0055PE,DP03_0056PE,DP03_0057PE,DP03_0058PE,DP03_0059PE,DP03_0060PE,DP03_0061PE,DP03_0062PE,DP03_0063PE,DP03_0064PE,DP03_0065PE,DP03_0066PE,DP03_0067PE,DP03_0068PE,DP03_0069PE,DP03_0070PE,DP03_0071PE,DP03_0072PE,DP03_0073PE,DP03_0074PE,DP03_0075PE,DP03_0076PE,DP03_0077PE,DP03_0078PE,DP03_0079PE,DP03_0080PE,DP03_0081PE,DP03_0082PE,DP03_0083PE,DP03_0084PE,DP03_0085PE,DP03_0086PE,DP03_0087PE,DP03_0088PE,DP03_0089PE,DP03_0090PE,DP03_0091PE,DP03_0092PE,DP03_0093PE,DP03_0094PE,DP03_0095PE,DP03_0096PE,DP03_0097PE,DP03_0098PE,DP03_0099PE,DP03_0100PE,DP03_0101PE,DP03_0102PE,DP03_0103PE,DP03_0104PE,DP03_0105PE,DP03_0106PE,DP03_0107PE,DP03_0108PE,DP03_0109PE,DP03_0110PE,DP03_0111PE,DP03_0112PE,DP03_0113PE,DP03_0114PE,DP03_0115PE,DP03_0116PE,DP03_0117PE,DP03_0118PE,DP03_0119PE,DP03_0120PE,DP03_0121PE,DP03_0122PE,DP03_0123PE,DP03_0124PE,DP03_0125PE,DP03_0126PE,DP03_0127PE,DP03_0128PE,DP03_0129PE,DP03_0130PE,DP03_0131PE,DP03_0132PE,DP03_0133PE,DP03_0134PE,DP03_0135PE,DP03_0136PE,DP03_0137PE,DP04_0001PE,DP04_0002PE,DP04_0003PE,DP04_0004PE,DP04_0005PE,DP04_0006PE,DP04_0007PE,DP04_0008PE,DP04_0009PE,DP04_0010PE,DP04_0011PE,DP04_0012PE,DP04_0013PE,DP04_0014PE,DP04_0015PE,DP04_0016PE,DP04_0017PE,DP04_0018PE,DP04_0019PE,DP04_0020PE,DP04_0021PE,DP04_0022PE,DP04_0023PE,DP04_0024PE,DP04_0025PE,DP04_0026PE,DP04_0027PE,DP04_0028PE,DP04_0029PE,DP04_0030PE,DP04_0031PE,DP04_0032PE,DP04_0033PE,DP04_0034PE,DP04_0035PE,DP04_0036PE,DP04_0037PE,DP04_0038PE,DP04_0039PE

In [171]:
pd.set_option('display.max_columns', None)
df_2016.loc[df_2016.state!="72"].apply(pd.to_numeric, errors="ignore").describe()

,AIANHH,state,county,AIHHTLI,AITSCE,ANRC,CBSA,CD115,CNECTA,CONCIT,COUNTY,COUSUB,CSA,DIVISION,DP02_0001PE,DP02_0002PE,DP02_0003PE,DP02_0004PE,DP02_0005PE,DP02_0006PE,DP02_0007PE,DP02_0008PE,DP02_0009PE,DP02_0010PE,DP02_0011PE,DP02_0012PE,DP02_0013PE,DP02_0014PE,DP02_0015PE,DP02_0016PE,DP02_0017PE,DP02_0018PE,DP02_0019PE,DP02_0020PE,DP02_0021PE,DP02_0022PE,DP02_0023PE,DP02_0024PE,DP02_0025PE,DP02_0026PE,DP02_0027PE,DP02_0028PE,DP02_0029PE,DP02_0030PE,DP02_0031PE,DP02_0032PE,DP02_0033PE,DP02_0034PE,DP02_0035PE,DP02_0036PE,DP02_0037PE,DP02_0038PE,DP02_0039PE,DP02_0040PE,DP02_0041PE,DP02_0042PE,DP02_0043PE,DP02_0044PE,DP02_0045PE,DP02_0046PE,DP02_0047PE,DP02_0048PE,DP02_0049PE,DP02_0050PE,DP02_0051PE,DP02_0052PE,DP02_0053PE,DP02_0054PE,DP02_0055PE,DP02_0056PE,DP02_0057PE,DP02_0058PE,DP02_0059PE,DP02_0060PE,DP02_0061PE,DP02_0062PE,DP02_0063PE,DP02_0064PE,DP02_0065PE,DP02_0066PE,DP02_0067PE,DP02_0068PE,DP02_0069PE,DP02_0070PE,DP02_0071PE,DP02_0072PE,DP02_0073PE,DP02_0074PE,DP02_0075PE,DP02_0076PE,DP02_0077PE,DP02_0078PE,DP02_0079PE,DP02_0080PE,DP02_0081PE,DP02_0082PE,DP02_0083PE,DP02_0084PE,DP02_0085PE,DP02_0086PE,DP02_0087PE,DP02_0088PE,DP02_0089PE,DP02_0090PE,DP02_0091PE,DP02_0092PE,DP02_0093PE,DP02_0094PE,DP02_0095PE,DP02_0096PE,DP02_0097PE,DP02_0098PE,DP02_0099PE,DP02_0100PE,DP02_0101PE,DP02_0102PE,DP02_0103PE,DP02_0104PE,DP02_0105PE,DP02_0106PE,DP02_0107PE,DP02_0108PE,DP02_0109PE,DP02_0110PE,DP02_0111PE,DP02_0112PE,DP02_0113PE,DP02_0114PE,DP02_0115PE,DP02_0116PE,DP02_0117PE,DP02_0118PE,DP02_0119PE,DP02_0120PE,DP02_0121PE,DP02_0122PE,DP02_0123PE,DP02_0124PE,DP02_0125PE,DP02_0126PE,DP02_0127PE,DP02_0128PE,DP02_0129PE,DP02_0130PE,DP02_0131PE,DP02_0132PE,DP02_0133PE,DP02_0134PE,DP02_0135PE,DP02_0136PE,DP02_0137PE,DP02_0138PE,DP02_0139PE,DP02_0140PE,DP02_0141PE,DP02_0142PE,DP02_0143PE,DP02_0144PE,DP02_0145PE,DP02_0146PE,DP02_0147PE,DP02_0148PE,DP02_0149PE,DP02_0150PE,DP02_0151PE,DP02_0152PE,DP03_0001PE,DP03_0002PE,DP03_0003PE,DP03_0004PE,DP03_0005PE,DP03_0006PE,DP03_0007PE,DP03_0008PE,DP03_0009PE,DP03_0010PE,DP03_0011PE,DP03_0012PE,DP03_0013PE,DP03_0014PE,DP03_0015PE,DP03_0016PE,DP03_0017PE,DP03_0018PE,DP03_0019PE,DP03_0020PE,DP03_0021PE,DP03_0022PE,DP03_0023PE,DP03_0024PE,DP03_0025PE,DP03_0026PE,DP03_0027PE,DP03_0028PE,DP03_0029PE,DP03_0030PE,DP03_0031PE,DP03_0032PE,DP03_0033PE,DP03_0034PE,DP03_0035PE,DP03_0036PE,DP03_0037PE,DP03_0038PE,DP03_0039PE,DP03_0040PE,DP03_0041PE,DP03_0042PE,DP03_0043PE,DP03_0044PE,DP03_0045PE,DP03_0046PE,DP03_0047PE,DP03_0048PE,DP03_0049PE,DP03_0050PE,DP03_0051PE,DP03_0052PE,DP03_0053PE,DP03_0054PE,DP03_0055PE,DP03_0056PE,DP03_0057PE,DP03_0058PE,DP03_0059PE,DP03_0060PE,DP03_0061PE,DP03_0062PE,DP03_0063PE,DP03_0064PE,DP03_0065PE,DP03_0066PE,DP03_0067PE,DP03_0068PE,DP03_0069PE,DP03_0070PE,DP03_0071PE,DP03_0072PE,DP03_0073PE,DP03_0074PE,DP03_0075PE,DP03_0076PE,DP03_0077PE,DP03_0078PE,DP03_0079PE,DP03_0080PE,DP03_0081PE,DP03_0082PE,DP03_0083PE,DP03_0084PE,DP03_0085PE,DP03_0086PE,DP03_0087PE,DP03_0088PE,DP03_0089PE,DP03_0090PE,DP03_0091PE,DP03_0092PE,DP03_0093PE,DP03_0094PE,DP03_0095PE,DP03_0096PE,DP03_0097PE,DP03_0098PE,DP03_0099PE,DP03_0100PE,DP03_0101PE,DP03_0102PE,DP03_0103PE,DP03_0104PE,DP03_0105PE,DP03_0106PE,DP03_0107PE,DP03_0108PE,DP03_0109PE,DP03_0110PE,DP03_0111PE,DP03_0112PE,DP03_0113PE,DP03_0114PE,DP03_0115PE,DP03_0116PE,DP03_0117PE,DP03_0118PE,DP03_0119PE,DP03_0120PE,DP03_0121PE,DP03_0122PE,DP03_0123PE,DP03_0124PE,DP03_0125PE,DP03_0126PE,DP03_0127PE,DP03_0128PE,DP03_0129PE,DP03_0130PE,DP03_0131PE,DP03_0132PE,DP03_0133PE,DP03_0134PE,DP03_0135PE,DP03_0136PE,DP03_0137PE,DP04_0001PE,DP04_0002PE,DP04_0003PE,DP04_0004PE,DP04_0005PE,DP04_0006PE,DP04_0007PE,DP04_0008PE,DP04_0009PE,DP04_0010PE,DP04_0011PE,DP04_0012PE,DP04_0013PE,DP04_0014PE,DP04_0015PE,DP04_0016PE,DP04_0017PE,DP04_0018PE,DP04_0019PE,DP04_0020PE,DP04_0021PE,DP04_0022PE,DP04_0023PE,DP04_0024PE,DP04_0025PE,DP04_0026PE,DP04_0027PE,DP04_0028PE,DP04_0029PE,DP04_0030PE,DP04_0031PE,DP04_0032PE,DP04_0033PE,DP04_0034PE,DP04_0035PE,DP04_0036PE,DP04_00

In [180]:
var_description.loc["DP02_0058PE"]

label    Percent Educational Attainment Population 25 Y...
Name: DP02_0058PE, dtype: object

In [179]:
pd.to_numeric(df_2016.loc[df_2016.state=="01", "DP02_0058PE"]) / pd.to_numeric(df_2016.loc[df_2016.state=="01", "DP02_0001PE"])

62     1.753173
63     1.862247
64     2.015676
65     2.234677
66     1.914351
67     2.062148
68     1.817459
69     1.745269
70     1.722186
71     1.699609
72     1.789217
73     1.698670
74     1.781662
75     1.761328
76     1.776826
77     1.786529
78     1.731328
79     1.730452
80     1.879933
81     1.745306
82     1.760229
83     1.812136
84     1.775194
85     1.679483
86     1.912055
87     1.902042
88     1.903221
89     1.814078
90     1.723504
91     1.789177
         ...   
99     1.663561
100    1.646151
101    1.758652
102    1.556087
103    1.934213
104    1.690656
105    1.568414
106    1.692552
107    1.678611
108    1.724972
109    1.838774
110    1.781999
111    1.790885
112    1.662277
113    1.776991
114    1.857724
115    1.848779
116    1.532417
117    1.794845
118    1.726951
119    1.886592
120    1.830568
121    1.697007
122    1.779759
123    1.769494
124    1.748367
125    1.811265
126    1.885163
127    1.962289
128    1.830649
Length: 67, dtype: float

In [133]:
df_2013 = reduce(lambda left, right: pd.merge(left, right, how='outer'), varlist_2013)

In [130]:
df_2015 = reduce(lambda left, right: pd.merge(left, right, how='outer'), varlist_2015).shape

In [127]:
df_2017.shape

(3220, 544)

In [124]:
df_2017 = reduce(lambda left, right: pd.merge(left, right, how='outer'), varlist_2017)

In [193]:
varlist_dict = {}
var_dict = {}
for year in range(2010, 2018):
    varlist_dict[year] = []
    k = 0
    for var in variables_selected.index:
        success = False
        l = 0
        while not success:
            l += 1
            resp = requests.get(
                "https://api.census.gov/data/{}/acs/acs5/profile?get={}&for=county:*&in=state:*&key={}"
                .format(year, var, censuskey))
            time.sleep(1)
            if resp.status_code ==200:
                varlist_dict[year].append(pd.DataFrame(resp.json()[1:], columns=resp.json()[0]))
                success = True
            elif l > 10:
                success = True
        k += 1
        
        if k%int(len(variables_selected)/10)==0:
            print("---progress: {:0.2f}% completed for {}---".format(
                k*10/int(len(variables_selected)/10), 
                year
            ))
    var_dict[year] = reduce(lambda left, right: pd.merge(left, 
                                                         right, 
                                                         how='outer'
                                                        ), varlist_dict[year])
    print(year, var_dict[year].shape)

---progress: 10.00% completed for 2010---
---progress: 20.00% completed for 2010---
---progress: 30.00% completed for 2010---
---progress: 40.00% completed for 2010---
---progress: 50.00% completed for 2010---
---progress: 60.00% completed for 2010---
---progress: 70.00% completed for 2010---
---progress: 80.00% completed for 2010---
---progress: 90.00% completed for 2010---
---progress: 100.00% completed for 2010---
2010 (3221, 544)
---progress: 10.00% completed for 2011---
---progress: 20.00% completed for 2011---
---progress: 30.00% completed for 2011---
---progress: 40.00% completed for 2011---
---progress: 50.00% completed for 2011---
---progress: 60.00% completed for 2011---
---progress: 70.00% completed for 2011---
---progress: 80.00% completed for 2011---
---progress: 90.00% completed for 2011---
---progress: 100.00% completed for 2011---
2011 (3221, 545)
---progress: 10.00% completed for 2012---
---progress: 20.00% completed for 2012---
---progress: 30.00% completed for 2012--

In [198]:
for year in range(2010, 2018):
    var_dict[year].to_csv("../data/census_{}.csv".format(year))

In [199]:
var_dict

552